## Notebook to take `blast` output from transcriptome v 3.1 to GOslim

### Set working directory

In [1]:
pwd

'/Users/graciecrandall/Documents/GitHub/paper-tanner-crab/notebooks'

In [2]:
wd = "../analyses/BLAST-to-GOslim/"

In [3]:
cd $wd

/Users/graciecrandall/Documents/GitHub/paper-tanner-crab/analyses/BLAST-to-GOslim


### Need three files in this directory

1. `blast` output from transcriptome v 3.1 in format -6
2. Uniprot GO annotation file (340M) available here: http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted
3. GOslim file available here: http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

In [4]:
#`curl` `blast` output
!curl --insecure https://gannet.fish.washington.edu/Atumefaciens/20200508_cbai_diamond_blastx_transcriptome-v2.0/20200507.C_bairdi.Trinity.blastx.outfmt6 \
-o 20200507.C_bairdi.Trinity.blastx.outfmt6

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.5M  100 12.5M    0     0  5082k      0  0:00:02  0:00:02 --:--:-- 7076k


In [5]:
!curl http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted -o uniprot-SP-GO.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  15.7M      0  0:00:21  0:00:21 --:--:-- 21.5M


In [6]:
!curl http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted -o GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0  1274k      0  0:00:01  0:00:01 --:--:-- 2385k


In [7]:
#check that files are in directory
!ls

20200507.C_bairdi.Trinity.blastx.outfmt6
GO-GOslim.sorted
GOslim-P-pie.txt
GOslim-pie-excel.pdf
GOslim-pie-transcriptome-v3.1.pdf
_blast-sep.tab
make-GOslim-pie-transcriptome-v3.1.xlsx
readme.md
uniprot-SP-GO.sorted


In [8]:
#set `blast` output file as a variable
blastout="20200507.C_bairdi.Trinity.blastx.outfmt6"

In [9]:
!head -2 $blastout

TRINITY_DN88428_c0_g1_i1	sp|Q9VVH9|SO74D_DROME	48.2	303	151	5	249	1157	154	450	1.6e-70	268.1
TRINITY_DN88425_c3_g1_i1	sp|P11309|PIM1_HUMAN	91.0	344	0	1	2328	1297	1	313	7.0e-183	642.5


In [10]:
#convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [11]:
!head -2 _blast-sep.tab
#commit _blast-sep.tab to directory --> will be used to get list of Trinity_IDs from GOslim 'stress response'

TRINITY_DN88428_c0_g1_i1	sp	Q9VVH9	SO74D_DROME	48.2	303	151	5	249	1157	154	450	1.6e-70	268.1
TRINITY_DN88425_c3_g1_i1	sp	P11309	PIM1_HUMAN	91.0	344	0	1	2328	1297	1	313	7.0e-183	642.5


In [12]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $3, $1, $13}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [13]:
!head -2 _blast-sort.tab

A0A024FA41	TRINITY_DN17783_c0_g1_i1	5.1e-05
A0A024SMV2	TRINITY_DN35734_c0_g1_i1	5.9e-18


In [14]:
!head -2 uniprot-SP-GO.sorted

A0A023GPI8	LECA_CANBL	reviewed	Lectin alpha chain (CboL) [Cleaved into: Lectin beta chain; Lectin gamma chain]		Canavalia boliviana	237			mannose binding [GO:0005537]; metal ion binding [GO:0046872]	mannose binding [GO:0005537]; metal ion binding [GO:0046872]	GO:0005537; GO:0046872
A0A023GPJ0	CDII_ENTCC	reviewed	Immunity protein CdiI	cdiI ECL_04450.1	Enterobacter cloacae subsp. cloacae (strain ATCC 13047 / DSM 30054 / NBRC 13535 / NCDC 279-56)	145					


In [15]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [16]:
!head -2 _blast-annot.tab

A0A024FA41	TRINITY_DN17783_c0_g1_i1	GO:0005506; GO:0008610; GO:0009405; GO:0016021; GO:0016491
A0A024SMV2	TRINITY_DN35734_c0_g1_i1	GO:0047837


#### The following is a script modified by Sam White

In [17]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [18]:
!head _blast-GO-unfolded.tab

A0A024FA41	TRINITY_DN17783_c0_g1_i1	GO:0005506
A0A024FA41	TRINITY_DN17783_c0_g1_i1	GO:0008610
A0A024FA41	TRINITY_DN17783_c0_g1_i1	GO:0009405
A0A024FA41	TRINITY_DN17783_c0_g1_i1	GO:0016021
A0A024FA41	TRINITY_DN17783_c0_g1_i1	GO:0016491
A0A024SMV2	TRINITY_DN35734_c0_g1_i1	GO:0047837
A0A075B6H9	TRINITY_DN230737_c0_g1_i1	GO:0002250
A0A075B6H9	TRINITY_DN230737_c0_g1_i1	GO:0002377
A0A075B6H9	TRINITY_DN230737_c0_g1_i1	GO:0003823
A0A075B6H9	TRINITY_DN230737_c0_g1_i1	GO:0005615


In [19]:
#gets rid of lines with no GOIDs
!sort -k3 _blast-GO-unfolded.tab | grep "GO:" > _blast-GO-unfolded.sorted

In [20]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $3, $1, $5, $6}' \
> Blastquery-GOslim.tab

In [21]:
!head Blastquery-GOslim.tab

TRINITY_DN18138_c0_g2_i1	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i1	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i10	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i12	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i2	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i3	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i5	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i6	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i7	GO:0000001	cell organization and biogenesis	P
TRINITY_DN12254_c0_g1_i9	GO:0000001	cell organization and biogenesis	P


In [23]:
!cat Blastquery-GOslim.tab | grep "	P"  | awk -F $'\t' '{print $4}' | sort | uniq -c

595541 P


In [24]:
!cat Blastquery-GOslim.tab | grep "	P" | awk -F $'\t' '{print $3}' | sort | uniq -c | sort -r

88398 other biological processes
70628 developmental processes
65914 cell organization and biogenesis
65145 other metabolic processes
63388 RNA metabolism
6030 cell adhesion
53012 protein metabolism
5234 cell-cell signaling
46892 transport
35631 signal transduction
35478 stress response
27506 cell cycle and proliferation
20463 DNA metabolism
11822 death


In [27]:
!cat Blastquery-GOslim.tab | grep "	P" | awk -F $'\t' '{print $3}' | sort | uniq -c | sort -r > GOslim-P-pie.txt